In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%pylab inline

import pandas as pd
import numpy as np
import warnings
from multiprocessing.dummy import Pool as ThreadPool 
import dateutil.parser
import nltk
import os
from nltk.corpus import stopwords
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.chunk import ne_chunk
from nltk.tag import pos_tag
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix 
from multiprocessing import Pool
from nltk import WordNetLemmatizer
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans, MiniBatchKMeans
from nltk.stem.lancaster import LancasterStemmer

warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


## Read in the text and clean the data

In [3]:
df = pd.read_csv("Complete_data.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44912 entries, 0 to 44911
Data columns (total 6 columns):
Unnamed: 0    44912 non-null int64
Date          44912 non-null object
From          44912 non-null object
To            44912 non-null object
Subject       44912 non-null object
Content       44457 non-null object
dtypes: int64(1), object(5)
memory usage: 2.1+ MB


In [5]:

x = list(range(0,len(df)-1))
for i in x:
    if "html" in str(df["Content"][i]):
        df.drop([i], inplace = True)

In [6]:
df.drop("Unnamed: 0",1,inplace = True)
df.fillna("Not Available", inplace = True)

In [7]:
#convert the dates in the emails to a datetime object
def to_date(datestring):
    date = dateutil.parser.parse(datestring)
    return date

df["Date"] = df["Date"].apply(to_date)

In [8]:
count = 0
for i in df["Content"]:
    if "html" in i:
        df = df.drop(df.index[count])
    count += 1

In [9]:
df["Content"] = df["Content"].apply(lambda x: x.replace("http", ''))
df["Content"] = df["Content"].apply(lambda x: x.replace(".com", ''))
df["Content"] = df["Content"].apply(lambda x: x.replace("www", ''))

In [10]:
common_html = ["blockquote", 'body', 'center', "del", 'div', 'font', 'head', ' hr ', 'block', 'align', '0px', '3d', 'arial', 'background', 'bgcolor', ' br ', 'cellpadding', 'cellspacing',
              'div', 'font', 'height', 'helvetica','href', 'img', 'valign', 'width', 'strong', 'serif', 'sans', ' alt ', 'display', 'src', 'style', ' tr ', 'tdtable', ' td ', 'tdtr', ' ef '
              'png', 'text', ' id ', 'gov', 'net']

for i in common_html:
    df["Content"] = df["Content"].apply(lambda x: x.replace(i, ''))

In [11]:
from nltk.stem.wordnet import WordNetLemmatizer #To download corpora: python -m    nltk.downloader all
lmtzr=WordNetLemmatizer()
df["Content"] = df["Content"].apply(lambda x: " ".join(lmtzr.lemmatize(word) for word in x.split(" ")))

In [12]:
text = []
for i in df["Content"]:
    text.append(i)

In [13]:
import pickle 

with open('text.pkl', 'wb') as f:
    pickle.dump(text,f)

In [14]:
len(text)

39639

In [15]:
count = 0
for i in text:
    for word in i:
        count += 1
count

93438257

In [16]:
vectorizer = CountVectorizer(min_df = 50, max_df = .1,stop_words = 'english')
dtm = vectorizer.fit_transform(text)

## Using LDA for topic modeling

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
from gensim import corpora, models, similarities, matutils 

In [18]:
doc_vecs = vectorizer.transform(text).transpose()
doc_vecs.shape

(9657, 39639)

In [19]:
corpus = matutils.Sparse2Corpus(doc_vecs)

In [20]:
id2word = dict((v, k) for k, v in vectorizer.vocabulary_.items())

In [21]:
len(id2word)

9657

In [22]:
lda = models.LdaMulticore(corpus, id2word=id2word, num_topics=20, passes=20)

In [24]:
lda.print_topics(num_words=10, num_topics=20)

[(0,
  '0.015*"00" + 0.011*"oct" + 0.009*"cell" + 0.008*"dinner" + 0.008*"wjc" + 0.007*"tina" + 0.007*"friday" + 0.007*"ny" + 0.006*"fran" + 0.006*"saturday"'),
 (1,
  '0.019*"health" + 0.016*"gun" + 0.015*"court" + 0.012*"care" + 0.012*"woman" + 0.011*"hangout" + 0.010*"supreme" + 0.008*"act" + 0.008*"site" + 0.007*"justice"'),
 (2,
  '0.027*"intended" + 0.018*"linkedin" + 0.016*"recipient" + 0.016*"confidential" + 0.016*"copy" + 0.016*"sender" + 0.015*"error" + 0.014*"communication" + 0.013*"attachment" + 0.012*"notify"'),
 (3,
  '0.020*"sander" + 0.017*"candidate" + 0.017*"republican" + 0.014*"democratic" + 0.013*"trump" + 0.013*"iowa" + 0.012*"voter" + 0.011*"democrat" + 0.011*"bernie" + 0.010*"vote"'),
 (4,
  '0.018*"donate" + 0.014*"united" + 0.013*"act" + 0.013*"contribution" + 0.011*"citizen" + 0.011*"donation" + 0.009*"chip" + 0.009*"election" + 0.009*"endcitizensunited" + 0.009*"supporter"'),
 (5,
  '0.060*"md5" + 0.045*"actblue" + 0.036*"contribute" + 0.034*"secure" + 0.030*

In [25]:
sorted(lda.show_topic(10, topn=10), key=lambda x: x[1], reverse=True) [:10]

[('link', 0.017648715565863487),
 ('washingtonpost', 0.015342111955968614),
 ('flint', 0.0144931768504082),
 ('la', 0.0099051870806396384),
 ('nh', 0.0092096088771816947),
 ('medium', 0.0081331370063462954),
 ('mayor', 0.0071915564926344749),
 ('photo', 0.006330740338487427),
 ('sc', 0.0062228890455594005),
 ('carolina', 0.0060668431592765647)]

In [26]:
sorted(lda.show_topic(1, topn=10), key=lambda x: x[1], reverse=True) [:10]

[('health', 0.018995189242584684),
 ('gun', 0.015730246963452255),
 ('court', 0.015379379846338574),
 ('care', 0.012370121600993451),
 ('woman', 0.011824201289791871),
 ('hangout', 0.011223293173734579),
 ('supreme', 0.010131898510740828),
 ('act', 0.0082532251485959388),
 ('site', 0.0080939500198908155),
 ('justice', 0.0073003484335017899)]

In [27]:
# Transform the docs from the word space to the topic space (like "transform" in sklearn)
lda_corpus = lda[corpus]

In [29]:
lda_docs = [doc for doc in lda_corpus]

In [33]:
len(lda_docs)

39639

In [28]:
np.save('lda.npy', lda)

In [30]:
np.save('lda_docs.npy',lda_docs)

In [74]:
topics_matrix = lda.show_topics(formatted=False, num_words=10, num_topics = 20)

In [75]:
len(topics_matrix)

20

In [76]:
topics = []
for i in topics_matrix:
    topics.append(i[1])

In [77]:
topics
topics_only = []
for i in topics:
    next_topic = []
    for element in i:
        next_topic.append(element[0])
    topics_only.append(next_topic)

In [78]:
for i in topics_only:
    print (i)

['00', 'oct', 'cell', 'dinner', 'wjc', 'tina', 'friday', 'ny', 'fran', 'saturday']
['health', 'gun', 'court', 'care', 'woman', 'hangout', 'supreme', 'act', 'site', 'justice']
['intended', 'linkedin', 'recipient', 'confidential', 'copy', 'sender', 'error', 'communication', 'attachment', 'notify']
['sander', 'candidate', 'republican', 'democratic', 'trump', 'iowa', 'voter', 'democrat', 'bernie', 'vote']
['donate', 'united', 'act', 'contribution', 'citizen', 'donation', 'chip', 'election', 'endcitizensunited', 'supporter']
['md5', 'actblue', 'contribute', 'secure', 'refcode', 'express', 'true', '25', 'recurring', 'honda']
['utm', 'energy', 'climate', 'content', 'link', 'clean', 'site', 'source', 'im', 'ct']
['size', 'span', 'class', 'table', 'color', 'margin', 'border', '100', 'knox', 'pub']
['school', 'community', 'university', 'program', 'research', 'education', 'project', 'manage', 'track', 'join']
['jake', 'jennifer', 'sullivan', 'palmieri', 'dan', 'robby', 'huma', 'grunwald', 'jim', 